In [64]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import os
import re
from pathlib import Path

In [161]:
class MergeandDistinct(object):
    
    def __init__(self,datapath,starttime1,starttime2,periods1,periods2,freqs):
        self.datapath=datapath
        self.starttime1=starttime1
        self.starttime2 = starttime2
        self.periods1 = periods1
        self.periods2 = periods2
        self.freqs = freqs
        self.newfilelist =[x.name for x in Path(self.datapath).iterdir() if x.is_dir()][1:]
        
    def minlimit(self):
        a = pd.DataFrame(pd.date_range(start = self.starttime1,periods = self.periods1,freq = self.freqs))
        b = pd.DataFrame(pd.date_range(start = self.starttime2,periods = self.periods2,freq = self.freqs))
        MinuteRange = pd.concat([a,b],axis =0)
        MinuteRange.columns=['Min']
        MinuteRange.reset_index(inplace = True)
        MinuteRange.Min = MinuteRange.Min.apply(lambda x: str(x)[11:] )
        MinuteRange.drop(columns = ['index'],inplace = True)
        return MinuteRange
    
    def datelimit(self):
        
        datelimList = []
        for i in range(len(self.newfilelist)): 
            lista = os.listdir(self.datapath+'/'+self.newfilelist[i])
            lista.sort(reverse = True)
            a = pd.read_csv(self.datapath+'/'+self.newfilelist[i]+'/'+lista[0]).iloc[-1]['date']
            datelimList.append(a)
        return datelimList
    
    def GetMinsData(self):
        Merge = []
        DLT = self.datelimit()
        MinuteRange=self.minlimit()
        for i in range(len(self.newfilelist)):
            try:
                lista = os.listdir(self.datapath+'/'+self.newfilelist[i])
                lista.sort(reverse = False)
                for ii in range(len(lista)):   
                    A = pd.read_csv(self.datapath+'/'+self.newfilelist[i]+'/'+lista[ii])
                    Merge.append(A)
                    if set(MinuteRange) == set(A['time']) and (DLT[i] in A['date']):
                        break 
            except: pass
        Result = pd.concat(Merge)        
        Result_Close = Result.groupby(['date','time']).head(1)[['date','time','spotclose']]
        Result_Close['date'] = pd.to_datetime(Result_Close.date)
        return Result_Close

In [166]:

A=MergeandDistinct("C:/Users/11604/Desktop/HS300min","09:30:00","13:00:00",121,121,"min")

In [167]:

A.GetMinsData()


date   time  spotclose
0    2020-01-02   9:30      4.121
1    2020-01-02   9:31      4.123
2    2020-01-02   9:33      4.126
3    2020-01-02   9:34      4.122
4    2020-01-02   9:35      4.123
...         ...    ...        ...
766  2020-04-15  13:54      3.794
1028 2020-04-20  13:56      3.827
66   2020-04-09  11:24      3.775
465  2020-04-08  13:01      3.765
581  2020-04-08  11:26      3.762

[18803 rows x 3 columns]